### **Data Annotation - Sentence paraphrasing**

After gathering and combining the scraped data into a single file, we used a paraphraser like "chatgpt_paraphraser_on_T5_base" from Hugging Face to create different versions of the questions we've collected. This helps in forming pairs of annotated questions. The paraphraser rephrases the original questions with varied wording and structure. For each original question, we generate five duplicate questions. We then pair the first two duplicates with the original question to form similar question pairs. The remaining three duplicates are shuffled to create a set of dissimilar questions.

In [1]:
# Import the pandas library as pd for data manipulation
import pandas as pd

# Read the CSV file containing the merged questions data into a DataFrame named df
df = pd.read_csv("data\merged_questions.csv")

In [9]:
# Paraphraser Reference: https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base

# Import necessary modules from the transformers library
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Set the device to use for processing
device = "cuda"

# Load the tokenizer from the pre-trained "chatgpt_paraphraser_on_T5_base" model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

# Load the model for sequence-to-sequence generation from the pre-trained "chatgpt_paraphraser_on_T5_base" model
# and move it to the specified device (GPU or CPU)
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define a function for paraphrasing input questions
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    # Tokenize the input question and prepare it for model input
    input_ids = tokenizer(
        f'paraphrase: {question}',  # Add prefix "paraphrase:" to indicate paraphrasing
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    # Generate paraphrases using the loaded model
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs and remove special tokens
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Return the paraphrased results
    return res

In [11]:
# Import tqdm library for progress tracking
import tqdm

# Ignore warnings to avoid cluttering the output
import warnings 
warnings.filterwarnings("ignore")

# Initialize an empty list to store paraphrased question pairs
l = []

# Intialy we did a sanity check, which run has already created paraphrases for first 3000 samples and were saved.
# Now we generate para[hrases for rest of the questions by looping over indices starting from 3000 until the length of the DataFrame
for ind in tqdm.tqdm(range(3000, len(df))):
    
    # Print progress message every 1000 iterations
    if (ind % 1000) == 0:
        print(ind)
        l = []
    
    try:
        # Get the text of the question from the DataFrame
        text = df.loc[ind, "q1"]
        
        # Generate paraphrases for the question text using the paraphrase function
        l.append([ind, text] + paraphrase(text))
    
    except:
        # Print an error message if an exception occurs during paraphrasing
        print("ERROR!!! @ ", ind)
    
    # Save the paraphrased question pairs to a CSV file every 1000 iterations
    if ((ind + 1) % 1000) == 0:
        d = pd.DataFrame(l, columns=["idx", "q1", "d1", "d2", "d3", "d4", "d5"])
        d.to_csv("pairs/pairs_{}.csv".format(ind))
    
    # Save the paraphrased question pairs to a CSV file when reaching the end of the DataFrame
    if ind == len(df):
        d = pd.DataFrame(l, columns=["idx", "q1", "d1", "d2", "d3", "d4", "d5"])
        d.to_csv("pairs/pairs_{}.csv".format(ind))

  0%|          | 0/17100 [00:00<?, ?it/s]

3000


  6%|▌         | 1000/17100 [11:02<2:52:06,  1.56it/s]

4000


 12%|█▏        | 2000/17100 [23:45<4:39:20,  1.11s/it]

...

 94%|█████████▎| 16000/17100 [2:48:01<12:44,  1.44it/s]  

19000


 99%|█████████▉| 17000/17100 [3:01:00<01:10,  1.41it/s]  

20000


100%|██████████| 17100/17100 [3:02:07<00:00,  1.56it/s]
